In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# 🤖 Step 0: Initialize the LLM with low randomness for reproducible output
llm = ChatOpenAI(
    temperature = 0.1
)

# 💾 Step 1: Set up caching for embeddings to save API costs and time
cache_dir = LocalFileStore("./.cache/")

# 📄 Step 2: Load and split the text document into overlapping chunks
loader = UnstructuredFileLoader("./files/chapter_one.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

# 🔍 Step 3: Embed the chunks using OpenAI (cached locally)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir
)

# 🧠 Step 4: Store embedded chunks in a FAISS vectorstore
vectorstore = FAISS.from_documents(docs, cached_embeddings)

print("✅ Vector store created with", len(docs), "chunks")

# 🔎 Step 5: Turn the vectorstore into a retriever
retriever = vectorstore.as_retriever()

# 🗺️ MAP STEP: Prompt to evaluate individual documents for relevance
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Use the following portion of a long document to determine if any of the text is relevant to answer the question. Return any relevant text verbatim.\n------\n{context}"),
    ("human", "{question}")
])

# 📦 Step 6: Chain to apply the map prompt across retrieved documents
map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    return "\n\n".join(map_doc_chain .invoke({
        "context": doc.page_content,
        "question": question
    }).content for doc in documents)

# 🧩 Step 7: Combine the map step with a retriever + question passthrough
map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

# 🧾 REDUCE STEP: Final prompt to answer based on all relevant chunks
final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Given the following extracted parts of a long document and a question, create a final answer.\n"
     "If you don't know the answer, just say that you don't know. Don't try to make up an answer.\n------\n{context}"),
    ("human", "{question}")
])

# 🔄 Final LCEL Chain: MAP (search + filter) ➜ REDUCE (summarize & answer)
chain = {"context": map_chain,"question": RunnablePassthrough()} | final_prompt | llm


# ❓ Step 8: Ask a question and run it through the full Map-Reduce pipeline
response = chain.invoke("Describe Victory Mansions")
print("🔍 Answer:", response.content)


## How it works with Map-Reduce LCEL CHAIN :)

# list of docs

# for doc in list of docs | prompt | llm

# for response in list of llm responses | put them all together

# final doc | prompt | llm

✅ Vector store created with 16 chunks


ValueError: Unexpected message type: hunan. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.